# **Model Version: 0.1**


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### ***Imports***

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import os
import numpy as np
import librosa
import random
from torch.utils.data import DataLoader
from torch import optim
import argparse
from sklearn.metrics import accuracy_score

### **Models**

In [ ]:
class TDNN(nn.Module):
    
    def __init__(
                    self, 
                    input_dim=23, 
                    output_dim=512,
                    context_size=5,
                    stride=1,
                    dilation=1,
                    batch_norm=False,
                    dropout_p=0.2
                ):
        '''
        TDNN as defined by https://www.danielpovey.com/files/2015_interspeech_multisplice.pdf

        Affine transformation not applied globally to all frames but smaller windows with local context

        batch_norm: True to include batch normalisation after the non linearity
        
        Context size and dilation determine the frames selected
        (although context size is not really defined in the traditional sense)
        For example:
            context size 5 and dilation 1 is equivalent to [-2,-1,0,1,2]
            context size 3 and dilation 2 is equivalent to [-2, 0, 2]
            context size 1 and dilation 1 is equivalent to [0]
        '''
        super(TDNN, self).__init__()
        self.context_size = context_size
        self.stride = stride
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dilation = dilation
        self.dropout_p = dropout_p
        self.batch_norm = batch_norm
      
        self.kernel = nn.Linear(input_dim*context_size, output_dim)
        self.nonlinearity = nn.ReLU()
        if self.batch_norm:
            self.bn = nn.BatchNorm1d(output_dim)
        if self.dropout_p:
            self.drop = nn.Dropout(p=self.dropout_p)
        
    def forward(self, x):
        '''
        input: size (batch, seq_len, input_features)
        outpu: size (batch, new_seq_len, output_features)
        '''
        
        _, _, d = x.shape
        assert (d == self.input_dim), 'Input dimension was wrong. Expected ({}), got ({})'.format(self.input_dim, d)
        x = x.unsqueeze(1)

        # Unfold input into smaller temporal contexts
        x = F.unfold(
                        x, 
                        (self.context_size, self.input_dim), 
                        stride=(1,self.input_dim), 
                        dilation=(self.dilation,1)
                    )

        # N, output_dim*context_size, new_t = x.shape
        x = x.transpose(1,2)
        x = self.kernel(x.float())
        x = self.nonlinearity(x)
        
        if self.dropout_p:
            x = self.drop(x)

        if self.batch_norm:
            x = x.transpose(1,2)
            x = self.bn(x)
            x = x.transpose(1,2)

        return x

In [ ]:
class X_vector(nn.Module):
    def __init__(self, input_dim = 40, num_classes=8):
        super(X_vector, self).__init__()
        self.tdnn1 = TDNN(input_dim=input_dim, output_dim=512, context_size=5, dilation=1,dropout_p=0.5)
        self.tdnn2 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=1,dropout_p=0.5)
        self.tdnn3 = TDNN(input_dim=512, output_dim=512, context_size=2, dilation=2,dropout_p=0.5)
        self.tdnn4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1,dropout_p=0.5)
        self.tdnn5 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=3,dropout_p=0.5)
        #### Frame levelPooling
        self.segment6 = nn.Linear(1024, 512)
        self.segment7 = nn.Linear(512, 512)
        self.output = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        tdnn1_out = self.tdnn1(inputs)
        return tdnn1_out
        tdnn2_out = self.tdnn2(tdnn1_out)
        tdnn3_out = self.tdnn3(tdnn2_out)
        tdnn4_out = self.tdnn4(tdnn3_out)
        tdnn5_out = self.tdnn5(tdnn4_out)
        ### Stat Pool
        mean = torch.mean(tdnn5_out,1)
        std = torch.std(tdnn5_out,1)
        stat_pooling = torch.cat((mean,std),1)
        segment6_out = self.segment6(stat_pooling)
        x_vec = self.segment7(segment6_out)
        predictions = self.softmax(self.output(x_vec))
        return predictions,x_vec

In [ ]:
class X_vector(nn.Module):
    def __init__(self, input_dim = 40, num_classes=8):
        super(X_vector, self).__init__()
        self.tdnn1 = TDNN(input_dim=input_dim, output_dim=512, context_size=5, dilation=1,dropout_p=0.5)
        self.tdnn2 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=1,dropout_p=0.5)
        self.tdnn3 = TDNN(input_dim=512, output_dim=512, context_size=2, dilation=2,dropout_p=0.5)
        self.tdnn4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1,dropout_p=0.5)
        self.tdnn5 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=3,dropout_p=0.5)
        #### Frame levelPooling
        self.segment6 = nn.Linear(1024, 512)
        self.segment7 = nn.Linear(512, 512)
        self.output = nn.Linear(512, num_classes)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, inputs):
        tdnn1_out = self.tdnn1(inputs)
        tdnn2_out = self.tdnn2(tdnn1_out)
        tdnn3_out = self.tdnn3(tdnn2_out)
        tdnn4_out = self.tdnn4(tdnn3_out)
        tdnn5_out = self.tdnn5(tdnn4_out)
        ### Stat Pool
        
        mean = torch.mean(tdnn5_out,1)
        std = torch.var(tdnn5_out,1)
        stat_pooling = torch.cat((mean,std),1)
        segment6_out = self.segment6(stat_pooling)
        x_vec = self.segment7(segment6_out)
        predictions = self.output(x_vec)
        return predictions,x_vec

### Utiles 

In [ ]:
def load_wav(audio_filepath, sr, min_dur_sec=4):
    audio_data,fs  = librosa.load(audio_filepath,sr=16000)
    len_file = len(audio_data)
    
    if len_file <int(min_dur_sec*sr):
        dummy=np.zeros((1,int(min_dur_sec*sr)-len_file))
        extened_wav = np.concatenate((audio_data,dummy[0]))
    else:
        
        extened_wav = audio_data
    return extened_wav


def lin_mel_from_wav(wav, hop_length, win_length, n_mels):
    linear = librosa.feature.melspectrogram(wav, n_mels=n_mels, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T

def lin_spectogram_from_wav(wav, hop_length, win_length, n_fft=512):
    linear = librosa.stft(wav, n_fft=n_fft, win_length=win_length, hop_length=hop_length) # linear spectrogram
    return linear.T


def feature_extraction(filepath,sr=16000, min_dur_sec=4,win_length=400,hop_length=160, n_mels=40, spec_len=400,mode='train'):
    audio_data = load_wav(filepath, sr=sr,min_dur_sec=min_dur_sec)
    linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_fft=512)
    mag, _ = librosa.magphase(linear_spect)  # magnitude
    mag_T = mag.T
    mu = np.mean(mag_T, 0, keepdims=True)
    std = np.std(mag_T, 0, keepdims=True)
    return (mag_T - mu) / (std + 1e-5)
    
    
    
    
def load_data(filepath,sr=16000, min_dur_sec=4,win_length=400,hop_length=160, n_mels=40, spec_len=400,mode='train'):
    audio_data = load_wav(filepath, sr=sr,min_dur_sec=min_dur_sec)
    #linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_mels)
    linear_spect = lin_spectogram_from_wav(audio_data, hop_length, win_length, n_fft=512)
    mag, _ = librosa.magphase(linear_spect)  # magnitude
    mag_T = mag.T
    
    if mode=='train':
        randtime = np.random.randint(0, mag_T.shape[1]-spec_len)
        spec_mag = mag_T[:, randtime:randtime+spec_len]
    else:
        spec_mag = mag_T
    
    # preprocessing, subtract mean, divided by time-wise var
    mu = np.mean(spec_mag, 0, keepdims=True)
    std = np.std(spec_mag, 0, keepdims=True)
    return (spec_mag - mu) / (std + 1e-5)
    


def load_npy_data(filepath,spec_len=400,mode='train'):
    mag_T = np.load(filepath)
    if mode=='train':
        randtime = np.random.randint(0, mag_T.shape[1]-spec_len)
        spec_mag = mag_T[:, randtime:randtime+spec_len]
    else:
        spec_mag = mag_T
    return spec_mag
    
    



def speech_collate(batch):
    targets = []
    specs = []
    for sample in batch:
        specs.append(sample['features'])
        targets.append((sample['labels']))
    return specs, targets

## SpeechDataGenerator

In [ ]:
class SpeechDataGenerator():
    """Speech dataset."""

    def __init__(self, manifest, mode):
        """
        Read the textfile and get the paths
        """
        
        # [line.rstrip('\n').split(' ')[0]
        self.mode=mode
        self.audio_links = [" ".join(line.rstrip('\n').split(' ')[:-1]) for line in open(manifest)]
        self.labels = [int(line.rstrip('\n').split(' ')[-1]) for line in open(manifest)]
        

    def __len__(self):
        return len(self.audio_links)

    def __getitem__(self, idx):
        audio_link =self.audio_links[idx]
        class_id = self.labels[idx]
        #lang_label=lang_id[self.audio_links[idx].split('/')[-2]]
        spec = load_data(audio_link,mode=self.mode)
        sample = {'features': torch.from_numpy(np.ascontiguousarray(spec)), 'labels': torch.from_numpy(np.ascontiguousarray(class_id))}
        return sample

## Train X Vector

In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

########## Argument parser

training_filepath = '/content/drive/My Drive/meta/training.txt'
testing_filepath = '/content/drive/My Drive/meta/testing.txt'
validation_filepath = '/content/drive/My Drive/meta/validation.txt'
input_dim = 257
num_classes = 9
lamda_val = 0.5
batch_size = 10
use_gpu = True
num_epochs = 30
# parser = argparse.ArgumentParser(add_help=False)
# parser.add_argument('-training_filepath', type=str, default='/content/drive/My Drive/meta/training.txt')
# parser.add_argument('-testing_filepath', type=str, default='/content/drive/My Drive/meta/testing.txt')
# parser.add_argument('-validation_filepath', type=str, default='/content/drive/My Drive/meta/validation.txt')

# parser.add_argument('-input_dim', action="store_true", default=257)
# parser.add_argument('-num_classes', action="store_true", default=9)
# parser.add_argument('-lamda_val', action="store_true", default=0.5)
# parser.add_argument('-batch_size', action="store_true", default=10)
# parser.add_argument('-use_gpu', action="store_true", default=True)
# parser.add_argument('-num_epochs', action="store_true", default=30)
#args = parser.parse_args()

### Data related
dataset_train = SpeechDataGenerator(manifest=training_filepath, mode='train')
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

dataset_val = SpeechDataGenerator(manifest=validation_filepath, mode='train')
dataloader_val = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

dataset_test = SpeechDataGenerator(manifest=testing_filepath, mode='test')
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=speech_collate)

## Model related
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = X_vector(input_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
loss_fun = nn.CrossEntropyLoss()


def train(dataloader_train, epoch):
    train_loss_list = []
    full_preds = []
    full_gts = []
    model.train()

    for i_batch, sample_batched in enumerate(dataloader_train):
        print("batch: " + str(i_batch+1))
        features = torch.from_numpy(np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
        labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]]))
        labels = labels.long()
        features, labels = features.to(device), labels.to(device)
        features.requires_grad = True
        optimizer.zero_grad()
        pred_logits, x_vec = model(features)
        #### CE loss
        loss = loss_fun(pred_logits, labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        # train_acc_list.append(accuracy)
        # if i_batch%10==0:
        #    print('Loss {} after {} iteration'.format(np.mean(np.asarray(train_loss_list)),i_batch))

        predictions = np.argmax(pred_logits.detach().cpu().numpy(), axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels.detach().cpu().numpy():
            full_gts.append(lab)

    mean_acc = accuracy_score(full_gts, full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss, mean_acc, epoch))


def validation(dataloader_val, epoch):
    model.eval()
    with torch.no_grad():
        val_loss_list = []
        full_preds = []
        full_gts = []
        for i_batch, sample_batched in enumerate(dataloader_val):
            features = torch.from_numpy(
                np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
            labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]]))
            labels = labels.long()
            features, labels = features.to(device), labels.to(device)
            pred_logits, x_vec = model(features)
            #### CE loss
            loss = loss_fun(pred_logits, labels)
            val_loss_list.append(loss.item())
            # train_acc_list.append(accuracy)
            predictions = np.argmax(pred_logits.detach().cpu().numpy(), axis=1)
            for pred in predictions:
                full_preds.append(pred)
            for lab in labels.detach().cpu().numpy():
                full_gts.append(lab)

        mean_acc = accuracy_score(full_gts, full_preds)
        mean_loss = np.mean(np.asarray(val_loss_list))
        print('Total validation loss {} and Validation accuracy {} after {} epochs'.format(mean_loss, mean_acc, epoch))

        # TODO: CHANGE BACK TO SAVE MODEL:
        # model_save_path = os.path.join('save_model', 'best_check_point_' + str(epoch) + '_' + str(mean_loss))
        # state_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
        # torch.save(state_dict, model_save_path)

In [ ]:
for epoch in range(num_epochs):
    print(epoch+1)
    train(dataloader_train, epoch)
    validation(dataloader_val, epoch)

1
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
Total training loss 1.966078437699212 and training Accuracy 0.21555555555555556 after 0 epochs
Total valida

## Data Generator 

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import random
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import split_on_silence
from concurrent.futures import ThreadPoolExecutor

In [ ]:
def transform_audio(audio_data, sr, n_steps=None, noise_mean=0., noise_std=None, speed_rate=None):
    if n_steps is not None:
        audio_data = librosa.effects.pitch_shift(audio_data, sr=sr, n_steps=n_steps)
    if noise_std is not None:
        noise = np.random.normal(noise_mean, noise_std, audio_data.shape)
        audio_data = np.clip(audio_data + noise, -1., 1.)
    if speed_rate is not None:
        audio_data = librosa.effects.time_stretch(audio_data, rate=speed_rate)
    return audio_data


def reduce_noise(audio_file_path):
    audio_file = AudioSegment.from_wav(audio_file_path)
    chunks = split_on_silence(audio_file, min_silence_len=1000, silence_thresh=-40)
    selected_chunks = [chunk for chunk in chunks if len(chunk) > 0]
    new_audio = selected_chunks[0]
    for chunk in selected_chunks[1:]:
        new_audio += chunk
    new_audio.export(audio_file_path, format='wav')


def convert_transform_and_reduce_noise(file_path):
    # Convert to WAV
    sound = AudioSegment.from_mp3(file_path)
    fname_without_ext = os.path.splitext(file_path)[0]
    wav_path = f"{fname_without_ext}.wav"
    sound.export(wav_path, format="wav")
    os.remove(file_path)  # delete original file

    # Reduce noise
    reduce_noise(wav_path)

    # Load WAV and transform audio
    audio_data, sr = librosa.load(wav_path, sr=None)
    audio_data_transformed = transform_audio(
        audio_data, sr, n_steps=0, noise_mean=0., noise_std=0.00, speed_rate=1
    )
    output_filename = f"{fname_without_ext}_Original.wav"
    sf.write(output_filename, audio_data_transformed, sr)

    random_float = round(random.uniform(0.00, 0.10), 2)
    audio_data_transformed_noise = transform_audio(
        audio_data, sr, n_steps=0, noise_mean=0., noise_std=random_float, speed_rate=1
    )
    output_filename = f"{fname_without_ext}_noise_{random_float}.wav"
    sf.write(output_filename, audio_data_transformed_noise, sr)

    random_float = round(random.uniform(1.0, 1.6), 2)
    audio_data_transformed_speed = transform_audio(
        audio_data, sr, n_steps=0, noise_mean=0., noise_std=0.00, speed_rate=random_float
    )
    output_filename = f"{fname_without_ext}_speed_{random_float}.wav"
    sf.write(output_filename, audio_data_transformed_speed, sr)

    random_float = round(random.uniform(1.0, 3.0), 2)
    audio_data_transformed_pitch = transform_audio(
        audio_data, sr, n_steps=random_float, noise_mean=0., noise_std=0.00, speed_rate=1
    )
    output_filename = f"{fname_without_ext}_pitch_{random_float}.wav"
    sf.write(output_filename, audio_data_transformed_pitch, sr)


    random_float = round(random.uniform(0.00, 0.10), 2)
    random_float1 = round(random.uniform(1.0, 1.6), 2)
    random_float2 = round(random.uniform(1.0, 3.0), 2)
    audio_data_transformed_all = transform_audio(
        audio_data, sr, n_steps=random_float, noise_mean=0., noise_std=0.00, speed_rate=1
    )
    output_filename = f"{fname_without_ext}_noise_{random_float}_speed_{random_float1}_pitch_{random_float2}.wav"
    sf.write(output_filename, audio_data_transformed_all, sr)

    return 1


def load(path):
    counter = 0
    mp3_files = []
    for dirName, subdirList, fileList in os.walk(path):
        print(f'Found directory: {dirName}')
        for fname in fileList:
            if fname.endswith('.mp3'):
                mp3_files.append(os.path.join(dirName, fname))

    with ThreadPoolExecutor() as executor:
        results = executor.map(convert_transform_and_reduce_noise, mp3_files)
        counter = sum(results)

    print('=============================')
    print(f'Total Data files loaded: {counter}')
    print('=============================')

In [ ]:
    rootDir = 'C:\\Users\\koazg\\Desktop\\same_size_lang'
    load(rootDir)